# Redes generativas

Las Redes Neuronales Recurrentes (RNNs) y sus variantes con celdas controladas, como las Celdas de Memoria a Largo Corto Plazo (LSTMs) y las Unidades Recurrentes Controladas (GRUs), proporcionaron un mecanismo para el modelado del lenguaje, es decir, pueden aprender el orden de las palabras y ofrecer predicciones para la siguiente palabra en una secuencia. Esto nos permite usar las RNNs para **tareas generativas**, como la generación de texto ordinario, la traducción automática e incluso la generación de subtítulos para imágenes.

En la arquitectura de RNN que discutimos en la unidad anterior, cada unidad RNN producía el siguiente estado oculto como salida. Sin embargo, también podemos añadir otra salida a cada unidad recurrente, lo que nos permitiría generar una **secuencia** (que tiene la misma longitud que la secuencia original). Además, podemos usar unidades RNN que no acepten una entrada en cada paso, y simplemente tomen un vector de estado inicial, para luego producir una secuencia de salidas.

En este cuaderno, nos centraremos en modelos generativos simples que nos ayuden a generar texto. Para simplificar, construyamos una **red a nivel de caracteres**, que genera texto letra por letra. Durante el entrenamiento, necesitamos tomar un corpus de texto y dividirlo en secuencias de letras.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Construcción de vocabulario de caracteres

Para construir una red generativa a nivel de caracteres, necesitamos dividir el texto en caracteres individuales en lugar de palabras. Esto se puede lograr definiendo un tokenizador diferente:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Veamos el ejemplo de cómo podemos codificar el texto de nuestro conjunto de datos:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Entrenando una RNN generativa

La forma en que entrenaremos la RNN para generar texto es la siguiente. En cada paso, tomaremos una secuencia de caracteres de longitud `nchars` y pediremos a la red que genere el siguiente carácter de salida para cada carácter de entrada:

![Imagen que muestra un ejemplo de generación de la palabra 'HELLO' con una RNN.](../../../../../lessons/5-NLP/17-GenerativeNetworks/images/rnn-generate.png)

Dependiendo del escenario específico, también podríamos querer incluir algunos caracteres especiales, como *fin de secuencia* `<eos>`. En nuestro caso, solo queremos entrenar la red para la generación continua de texto, por lo que fijaremos el tamaño de cada secuencia para que sea igual a `nchars` tokens. En consecuencia, cada ejemplo de entrenamiento consistirá en `nchars` entradas y `nchars` salidas (que son la secuencia de entrada desplazada un símbolo hacia la izquierda). El minibatch consistirá en varias de estas secuencias.

La forma en que generaremos los minibatches será tomando cada texto de noticias de longitud `l` y generando todas las combinaciones posibles de entrada-salida a partir de él (habrá `l-nchars` de estas combinaciones). Estas formarán un minibatch, y el tamaño de los minibatches será diferente en cada paso de entrenamiento.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Ahora definamos la red generadora. Puede basarse en cualquier célula recurrente que discutimos en la unidad anterior (simple, LSTM o GRU). En nuestro ejemplo, utilizaremos LSTM.

Dado que la red toma caracteres como entrada y el tamaño del vocabulario es bastante pequeño, no necesitamos una capa de embeddings; la entrada codificada en one-hot puede ir directamente a la célula LSTM. Sin embargo, como pasamos números de caracteres como entrada, necesitamos codificarlos en one-hot antes de pasarlos a la LSTM. Esto se realiza llamando a la función `one_hot` durante el paso `forward`. El codificador de salida sería una capa lineal que convertirá el estado oculto en una salida codificada en one-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Durante el entrenamiento, queremos poder muestrear texto generado. Para ello, definiremos la función `generate` que producirá una cadena de salida de longitud `size`, comenzando desde la cadena inicial `start`.

El funcionamiento es el siguiente. Primero, pasaremos toda la cadena inicial a través de la red, y obtendremos el estado de salida `s` y el siguiente carácter predicho `out`. Dado que `out` está codificado en one-hot, tomamos `argmax` para obtener el índice del carácter `nc` en el vocabulario, y usamos `itos` para identificar el carácter real y añadirlo a la lista resultante de caracteres `chars`. Este proceso de generar un carácter se repite `size` veces para generar el número requerido de caracteres.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

¡Ahora vamos a entrenar! El bucle de entrenamiento es casi el mismo que en todos nuestros ejemplos anteriores, pero en lugar de la precisión, imprimimos texto generado como muestra cada 1000 épocas.

Se debe prestar especial atención a la forma en que calculamos la pérdida. Necesitamos calcular la pérdida dado un resultado codificado en one-hot `out` y el texto esperado `text_out`, que es la lista de índices de caracteres. Por suerte, la función `cross_entropy` espera como primer argumento la salida no normalizada de la red, y como segundo argumento el número de clase, que es exactamente lo que tenemos. Además, realiza un promedio automático sobre el tamaño del minibatch.

También limitamos el entrenamiento a `samples_to_train` muestras, para no tener que esperar demasiado tiempo. Te animamos a experimentar y probar entrenamientos más largos, posiblemente durante varias épocas (en cuyo caso necesitarías crear otro bucle alrededor de este código).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Este ejemplo ya genera un texto bastante bueno, pero se puede mejorar en varios aspectos:

* **Mejor generación de minibatches**. La forma en que preparamos los datos para el entrenamiento fue generando un minibatch a partir de una muestra. Esto no es ideal, porque los minibatches tienen tamaños diferentes, y algunos incluso no pueden generarse porque el texto es más pequeño que `nchars`. Además, los minibatches pequeños no aprovechan suficientemente la GPU. Sería más inteligente tomar un gran bloque de texto de todas las muestras, luego generar todos los pares de entrada-salida, mezclarlos y generar minibatches de tamaño uniforme.

* **LSTM multicapa**. Tiene sentido probar con 2 o 3 capas de células LSTM. Como mencionamos en la unidad anterior, cada capa de LSTM extrae ciertos patrones del texto, y en el caso de un generador a nivel de caracteres, podemos esperar que el nivel inferior de LSTM sea responsable de extraer sílabas, y los niveles superiores de palabras y combinaciones de palabras. Esto se puede implementar fácilmente pasando el parámetro de número de capas al constructor de LSTM.

* También podrías experimentar con **unidades GRU** y ver cuáles funcionan mejor, así como con **diferentes tamaños de capas ocultas**. Una capa oculta demasiado grande puede resultar en sobreajuste (por ejemplo, la red aprenderá el texto exacto), mientras que un tamaño más pequeño podría no producir buenos resultados.


## Generación de texto suave y temperatura

En la definición anterior de `generate`, siempre tomábamos el carácter con la probabilidad más alta como el siguiente carácter en el texto generado. Esto daba como resultado que el texto a menudo "ciclaba" entre las mismas secuencias de caracteres una y otra vez, como en este ejemplo:
```
today of the second the company and a second the company ...
```

Sin embargo, si observamos la distribución de probabilidad para el siguiente carácter, podría suceder que la diferencia entre algunas de las probabilidades más altas no sea muy grande, por ejemplo, un carácter puede tener una probabilidad de 0.2, otro de 0.19, etc. Por ejemplo, al buscar el siguiente carácter en la secuencia '*play*', el siguiente carácter podría ser igualmente un espacio o una **e** (como en la palabra *player*).

Esto nos lleva a la conclusión de que no siempre es "justo" seleccionar el carácter con mayor probabilidad, ya que elegir el segundo más probable aún podría conducir a un texto significativo. Es más sensato **muestrear** caracteres de la distribución de probabilidad dada por la salida de la red.

Este muestreo se puede realizar utilizando la función `multinomial`, que implementa la llamada **distribución multinomial**. A continuación, se define una función que implementa esta generación de texto **suave**:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Hemos introducido un parámetro más llamado **temperatura**, que se utiliza para indicar qué tan estrictamente debemos adherirnos a la probabilidad más alta. Si la temperatura es 1.0, hacemos un muestreo multinomial justo, y cuando la temperatura se acerca al infinito, todas las probabilidades se vuelven iguales y seleccionamos el siguiente carácter al azar. En el ejemplo a continuación, podemos observar que el texto se vuelve sin sentido cuando aumentamos demasiado la temperatura, y se asemeja a un texto "cíclico" generado rígidamente cuando se acerca a 0.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
